<a href="https://colab.research.google.com/github/alerotta/DRL/blob/main/02%20-%20Q%20Learning/Q_Learning_FrozenLake_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gymnasium[toy-text] --quiet

In [2]:
import typing as tt
import gymnasium as gym
from collections import defaultdict, Counter
from torch.utils.tensorboard.writer import SummaryWriter

In [10]:
# this is Q-Learining and not Deep Q... so there is no use of n.n

ENV_NAME = "FrozenLake-v1"
TEST_EPISODES = 20
GAMMA = 0.9 # reward discount factor

State = int
Action = int
RewardKey = tt.Tuple[State, Action, State] # s, a, s'
TransitKey = tt.Tuple[State, Action] # s, a

class Agent:
  def __init__(self):
    self.env = gym.make(ENV_NAME) #create the environment
    self.state, _ = self.env.reset() #first state
    self.rewards: tt.Dict[RewardKey,float] = defaultdict(float) #given s,a,s' save reward
    self.transit: tt.Dict[TransitKey,Counter] = defaultdict(Counter)
    self.values: tt.Disc[TransitKey,float] = defaultdict(float)

  def play_n_random_steps(self,n):
    for _ in range(n):
      action = self.env.action_space.sample()
      new_state, reward, is_done ,is_trunc, _ = self.env.step(action)
      rw_key = (self.state , action, new_state)
      self.rewards[rw_key] = float(reward)
      tr_key = (self.state, action)
      self.transit[tr_key][new_state] += 1
      if is_done or is_trunc :
        self.state, _ = self.env.reset()
      else:
        self.state = new_state

  def select_action(self,state) -> Action :
    best_action, best_value = None, None
    for action in range(self.env.action_space.n):
      action_value = self.values[(state,action)]
      if best_value is None or best_value < action_value:
        best_value = action_value
        best_action = action

    return best_action


  def play_episode(self, env):
    total_reward = 0.0
    state, _ = env.reset()
    while True:
      action = self.select_action(state)
      new_state, reward , is_done, is_trunc , _ = env.step(action)
      rw_key = (state,action,new_state)
      self.rewards[rw_key]= float(reward)
      tr_key = (state,action)
      self.transit[tr_key][new_state] += 1
      total_reward += reward
      if is_done or is_trunc :
        break
      state = new_state
    return total_reward

  def value_iteration(self):
    for state in range(self.env.observation_space.n):
      for action in range(self.env.action_space.n):
        action_value = 0.0
        target_counts = self.transit[(state,action)]
        total = sum(target_counts.values())
        for tgt_state, count in target_counts.items():
          rw_key = (state,action,tgt_state)
          reward = self.rewards[rw_key]
          best_action = self.select_action(tgt_state)
          val = reward + GAMMA * self.values[(tgt_state,best_action)]
          action_value += (count/total)*val
        self.values [(state,action)] = action_value

def run():
    test_env = gym.make(ENV_NAME)
    agent = Agent()
    writer = SummaryWriter(comment="-q-iteration")

    iter_no = 0
    best_reward = 0.0
    while True:
        iter_no += 1
        agent.play_n_random_steps(100)
        agent.value_iteration()

        reward = 0.0
        for _ in range(TEST_EPISODES):
            reward += agent.play_episode(test_env)
        reward /= TEST_EPISODES
        writer.add_scalar("reward", reward, iter_no)
        if reward > best_reward:
            print(f"{iter_no}: Best reward updated "
                  f"{best_reward:.3} -> {reward:.3}")
            best_reward = reward
        if reward > 0.80:
            print("Solved in %d iterations!" % iter_no)
            break
    writer.close()
run()






14: Best reward updated 0.0 -> 0.35
17: Best reward updated 0.35 -> 0.55
24: Best reward updated 0.55 -> 0.75
42: Best reward updated 0.75 -> 0.8
43: Best reward updated 0.8 -> 0.85
Solved in 43 iterations!
